In [ ]:
# Quick test: does 4-bit quantization reproduce paper results
# Paper: BASE ~49%, AT ~43%, LAT ~51% top-1 variance

import torch
import numpy as np
import requests
import pandas as pd
import io
import gc
from scipy.linalg import svd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformer_lens import HookedTransformer
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from huggingface_hub import login as hf_login
import os
from dotenv import load_dotenv

load_dotenv()
hf_login(token=os.environ.get('HF_TOKEN'))


In [ ]:
DEVICE = 'cuda'
DTYPE = torch.float16
LLAMA_PATH = 'meta-llama/Llama-2-7b-chat-hf'
LAT_PATH = 'nlpett/llama-2-7b-chat-hf-LAT-layer4-hh'
AT_PATH = 'nlpett/llama-2-7b-chat-hf-AT-hh'
TEMPLATE = '\n[INST]{prompt}[/INST]\n'

# 4-bit config from original paper code
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=DTYPE,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)


In [ ]:
def load_model_4bit(peft_path=None, name='base'):
    base = AutoModelForCausalLM.from_pretrained(
        LLAMA_PATH, torch_dtype=DTYPE, device_map='cuda:0', quantization_config=bnb_config
    )
    if peft_path:
        peft = PeftModel.from_pretrained(base, peft_path).to(DTYPE)
        merged = peft.merge_and_unload()
        merged.eval()
        hf_model = merged
    else:
        hf_model = base
    
    hooked = HookedTransformer.from_pretrained_no_processing(
        LLAMA_PATH, hf_model=hf_model, dtype=DTYPE, device=DEVICE, default_padding_side='left'
    )
    hooked.tokenizer.padding_side = 'left'
    hooked.tokenizer.pad_token = '[PAD]'
    
    del base
    if peft_path:
        del peft, merged
    gc.collect()
    torch.cuda.empty_cache()
    return hooked


In [ ]:
# load data
url = 'https://raw.githubusercontent.com/llm-attacks/llm-attacks/main/data/advbench/harmful_behaviors.csv'
harmful_train, _ = train_test_split(
    pd.read_csv(io.StringIO(requests.get(url).content.decode('utf-8')))['goal'].tolist(), 
    test_size=0.2, random_state=42
)

dataset = load_dataset('tatsu-lab/alpaca')
harmless_all = [dataset['train'][i]['instruction'] for i in range(len(dataset['train'])) 
                if dataset['train'][i]['input'].strip() == '']
harmless_train, _ = train_test_split(harmless_all, test_size=0.2, random_state=42)

print(f'harmful: {len(harmful_train)}, harmless: {len(harmless_train)}')


In [ ]:
def get_acts(model, instructions, layer=14, batch_size=16):
    acts = []
    for i in tqdm(range(0, len(instructions), batch_size)):
        batch = instructions[i:i+batch_size]
        prompts = [TEMPLATE.format(prompt=p) for p in batch]
        toks = model.tokenizer(prompts, padding=True, return_tensors='pt').input_ids.to(DEVICE)
        with torch.no_grad():
            _, cache = model.run_with_cache(toks, names_filter=lambda n: f'blocks.{layer}.hook_resid_pre' in n)
        acts.append(cache[f'blocks.{layer}.hook_resid_pre'][:, -1, :])
    return torch.cat(acts, dim=0)

def svd_variance(diffs):
    _, s, _ = svd(diffs.cpu().numpy(), full_matrices=False)
    v = (s**2) / np.sum(s**2)
    return v[0], sum(v[:2]), sum(v[:10])


In [ ]:
N = 416
results = {}

for name, peft_path in [('BASE', None), ('AT', AT_PATH), ('LAT', LAT_PATH)]:
    print(f'\n=== {name} ===')
    model = load_model_4bit(peft_path, name)
    
    harmful_acts = get_acts(model, harmful_train[:N])
    harmless_acts = get_acts(model, harmless_train[:N])
    diffs = harmful_acts - harmless_acts
    
    top1, top2, top10 = svd_variance(diffs)
    results[name] = {'top1': top1, 'top2': top2, 'top10': top10}
    print(f'{name}: top-1={top1*100:.1f}%, top-2={top2*100:.1f}%, top-10={top10*100:.1f}%')
    
    del model, harmful_acts, harmless_acts, diffs
    gc.collect()
    torch.cuda.empty_cache()


In [ ]:
print(f"{'Model':<8} {'4bit top-1':>12} {'Paper top-1':>12} {'Match?':>10}")
print('-' * 44)
paper = {'BASE': 49, 'AT': 43, 'LAT': 51}
for name in ['BASE', 'AT', 'LAT']:
    got = results[name]['top1']*100
    expected = paper[name]
    match = '✓' if abs(got - expected) < 10 else '✗'
    print(f"{name:<8} {got:>11.1f}% {expected:>11}% {match:>10}")
